In [125]:
import sys
import gensim
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from string import punctuation as punct
from sklearn.cluster import AffinityPropagation, SpectralClustering
from sklearn.preprocessing import StandardScaler
from nltk import word_tokenize
import torch 

stops = stopwords.words('russian')
punct+=' –'

In [8]:
from tqdm import tqdm_notebook
from tqdm.auto import tqdm
tqdm_notebook().pandas()

/Users/t.zhordaniya/venv/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


/Users/t.zhordaniya/venv/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


## Word2Vec



In [33]:
path='182/model.bin.gz'
word2vec = gensim.models.KeyedVectors.load_word2vec_format(path, binary=True)

In [3]:
mapping={'A': 'ADJ', 'ADV': 'ADV', 'ADVPRO': 'ADV', 'ANUM': 'ADJ', 'APRO': 'DET', 'COM': 'ADJ', 'CONJ': 'SCONJ', 'INTJ': 'INTJ', 'NONLEX': 'X', 'NUM': 'NUM', 'PART': 'PART', 'PR': 'ADP', 'S': 'NOUN', 'SPRO': 'PRON', 'UNKN': 'X', 'V': 'VERB'}

In [20]:
from pymystem3 import Mystem
m = Mystem()

def tag_mystem(text):  
    
    processed = m.analyze(text)
    tagged = []
    for w in processed:
        try:
            lemma = w["analysis"][0]["lex"].lower().strip()
            pos = w["analysis"][0]["gr"].split(',')[0]
            pos = pos.split('=')[0].strip()
            if pos in mapping:
                tagged.append(lemma + '_' + mapping[pos]) 
            else:
                tagged.append(lemma + '_X') 
        except:
            continue 
    return ' '.join(tagged)

In [6]:
df_train = pd.read_csv('data/main/active-dict/train.csv', sep='\t')

In [15]:
df_train['context_tagged'] = df_train['context'].progress_apply(tag_mystem)

In [28]:
def find_tagret(x):
    try: 
        return [word for word in x.context_lem.split() if word.split('_')[0]==x.word][0]
    except:
        return tag_mystem(x.word)
    
def del_stops(context):
    
    w = [word for word in context.split() if word.split('_')[0] not in stops]
    return ' '.join(w)

In [23]:
df_train['target'] = df_train.progress_apply(find_tagret, axis=1)

In [29]:
df_train['context_cleaned'] = df_train['context_tagged'].progress_apply(del_stops)

In [30]:
df_train

,context_id,word,gold_sense_id,predict_sense_id,positions,context,context_tagged,target,context_cleaned
0,1,дар,1,NaN,18-22,Отвергнуть щедрый дар,отвергать_VERB щедрый_ADJ дар_NOUN,дар_NOUN,отвергать_VERB щедрый_ADJ дар_NOUN
1,2,дар,1,NaN,21-28,покупать преданность дарами и наградами,покупать_VERB преданность_NOUN дар_NOUN и_SCON...,дар_NOUN,покупать_VERB преданность_NOUN дар_NOUN наград...
2,3,дар,1,NaN,19-23,Вот яд – последний дар моей Изоры,вот_PART яд_NOUN последний_ADJ дар_NOUN мой_DE...,дар_NOUN,яд_NOUN последний_ADJ дар_NOUN изор_NOUN
3,4,дар,1,NaN,81-87,Основная функция корильных песен – повеселить ...,основной_ADJ функция_NOUN корильный_ADJ песня_...,дар_NOUN,основной_ADJ функция_NOUN корильный_ADJ песня_...
4,5,дар,1,NaN,151-157,Но недели две спустя (Алевтина его когда-то об...,но_SCONJ неделя_NOUN два_NUM спустя_ADP алевти...,дар_NOUN,неделя_NOUN спустя_ADP алевтина_NOUN когда-то_...
...,...,...,...,...,...,...,...,...,...
2068,2069,зонт,1,NaN,85-91,"Такая погода легко переживается весной, а вот ...",такой_DET погода_NOUN легко_ADV переживаться_V...,зонт_NOUN,погода_NOUN легко_ADV переживаться_VERB весна_...
2069,2070,зонт,2,NaN,8-13,Пляжный зонт,пляжный_ADJ зонт_NOUN,зонт_NOUN,пляжный_ADJ зонт_NOUN
2070,2071,зонт,2,NaN,18-25,сидеть в кафе под зонтом,сидеть_VERB в_ADP кафе_NOUN под_ADP зонт_NOUN,зонт_NOUN,сидеть_VERB кафе_NOUN зонт_NOUN
2071,2072,зонт,2,NaN,21-29,"Cтолики под широкими зонтами, несколько привин...",столик_NOUN под_ADP широкий_ADJ зонт_NOUN неск...,зонт_NOUN,столик_NOUN широкий_ADJ зонт_NOUN несколько_NU...


In [54]:
def average_vectors(x):
    vectors=[]
    context = x.context_cleaned
    words = list(set([word for word in context.split() if word != x.target and word in word2vec]))
    if not words:
        words = [x.target]
    
    for i, word in enumerate(words):
        
        vectors.append(word2vec[word])

    
    return np.mean(vectors, axis=0)
        
    

In [70]:

def predict_wor2vec(df):
    predicted = []

    for word in df['word'].unique():

        word_df = df[df['word'] == word]


        vectors = word_df.apply(average_vectors, axis=1) 

        matrix=np.vstack(vectors)

        clustering = AffinityPropagation(preference=-0.7, damping=0.7).fit(matrix)
        nclusters = len(clustering.cluster_centers_indices_)

        if nclusters < 1:
            nclusters = 1
        elif nclusters == len(word_df):
            nclusters = 4

        clustering = SpectralClustering(n_clusters=nclusters, n_init=20,
                                        assign_labels='discretize', n_jobs=2).fit(matrix)

        cur_predicted = clustering.labels_.tolist()
        predicted += cur_predicted
        
    return predicted

In [71]:
df_train['predict_sense_id'] = predict_wor2vec(df_train)

In [73]:
df_train.to_csv('df_train.csv', index=False, sep='\t')

In [80]:
!python3 evaluate.py df_train.csv

word	ari	count
дар	0.108842	36
двигатель	0.188742	15
двойник	0.096693	25
дворец	0.190661	13
девятка	0.135668	47
дедушка	0.100000	9
дежурная	0.050139	12
дежурный	0.122699	13
декабрист	0.485981	11
декрет	0.554913	12
дело	0.011877	130
демобилизация	0.548163	14
демократ	-0.030222	18
демонстрация	0.099838	38
дерево	0.552239	21
держава	0.171053	15
дерзость	0.087616	37
десятка	0.102005	36
десяток	0.080195	21
деятель	0.494700	14
диалог	0.313380	14
диаметр	0.194030	18
диплом	0.226384	25
директор	0.044118	11
диск	0.168480	63
дичь	0.491887	18
длина	-0.074024	21
доброволец	0.250710	12
добыча	0.212836	35
доказательство	0.139383	24
доктор	0.276596	17
долгота	0.011696	13
доля	0.089024	45
дом	0.137362	38
дорога	0.019604	47
достижение	0.068511	22
древесина	0.076133	16
дупло	0.731065	15
дура	0.693593	12
дух	0.102564	77
дым	0.499006	28
дымка	0.520376	18
дыхание	0.123509	56
дьявол	0.205078	22
евро	0.166667	8
езда	-0.053038	14
жаворонок	0.236111	11
жало	0.000000	11
жертва	0.005011	37
жестокость	0.001950	14

#### Проверим алгоритм на корпусе 

In [75]:
df_test = pd.read_csv('data/additional/active-rutenten/train.csv', sep='\t')

In [76]:
df_test['context_tagged'] = df_test['context'].progress_apply(tag_mystem)
df_test['target'] = df_test.progress_apply(find_tagret, axis=1)
df_test['context_cleaned'] = df_test['context_tagged'].progress_apply(del_stops)

In [77]:
df_test['predict_sense_id'] = predict_wor2vec(df_test)

In [78]:
df_test.to_csv('df_test.csv', index=False, sep='\t')

In [79]:
!python3 evaluate.py df_test.csv

word	ari	count
альбом	0.048016	450
анатомия	0.007360	95
базар	0.048682	90
балет	0.114912	94
беда	0.047208	93
бездна	0.013397	87
билет	-0.010174	447
блок	0.272103	206
блоха	-0.004986	86
брак	0.033178	96
бритва	0.004670	85
будущее	-0.005635	83
вешалка	0.059043	390
вилка	0.374105	302
винт	0.230417	358
галерея	0.169550	24
горбуша	0.520626	93
горшок	0.167009	406
гроза	-0.041698	95
группа	0.128040	91
	0.120870	3671


In [88]:
df_train

,context_id,word,gold_sense_id,predict_sense_id,positions,context,context_tagged,target,context_cleaned
0,1,дар,1,3,18-22,Отвергнуть щедрый дар,отвергать_VERB щедрый_ADJ дар_NOUN,дар_NOUN,отвергать_VERB щедрый_ADJ дар_NOUN
1,2,дар,1,0,21-28,покупать преданность дарами и наградами,покупать_VERB преданность_NOUN дар_NOUN и_SCON...,дар_NOUN,покупать_VERB преданность_NOUN дар_NOUN наград...
2,3,дар,1,0,19-23,Вот яд – последний дар моей Изоры,вот_PART яд_NOUN последний_ADJ дар_NOUN мой_DE...,дар_NOUN,яд_NOUN последний_ADJ дар_NOUN изор_NOUN
3,4,дар,1,0,81-87,Основная функция корильных песен – повеселить ...,основной_ADJ функция_NOUN корильный_ADJ песня_...,дар_NOUN,основной_ADJ функция_NOUN корильный_ADJ песня_...
4,5,дар,1,0,151-157,Но недели две спустя (Алевтина его когда-то об...,но_SCONJ неделя_NOUN два_NUM спустя_ADP алевти...,дар_NOUN,неделя_NOUN спустя_ADP алевтина_NOUN когда-то_...
...,...,...,...,...,...,...,...,...,...
2068,2069,зонт,1,2,85-91,"Такая погода легко переживается весной, а вот ...",такой_DET погода_NOUN легко_ADV переживаться_V...,зонт_NOUN,погода_NOUN легко_ADV переживаться_VERB весна_...
2069,2070,зонт,2,0,8-13,Пляжный зонт,пляжный_ADJ зонт_NOUN,зонт_NOUN,пляжный_ADJ зонт_NOUN
2070,2071,зонт,2,3,18-25,сидеть в кафе под зонтом,сидеть_VERB в_ADP кафе_NOUN под_ADP зонт_NOUN,зонт_NOUN,сидеть_VERB кафе_NOUN зонт_NOUN
2071,2072,зонт,2,1,21-29,"Cтолики под широкими зонтами, несколько привин...",столик_NOUN под_ADP широкий_ADJ зонт_NOUN неск...,зонт_NOUN,столик_NOUN широкий_ADJ зонт_NOUN несколько_NU...


## А что с бертом?

In [81]:
from transformers import BertTokenizer, BertModel

### Интуиция:



1) получить контекстно-зависимый эмбеддинг всего предложения

2) по индексу найти нужное нам слово

3) усреднить эмбеддинги словев этого слова (последние 4 слоя)


In [ ]:
df_train['positions'].isnull().values.any()

In [ ]:
len(df_train[df_train['positions'].isnull()]) # несколько битых строк, можно от них избавиться

In [ ]:
df_train = df_train[df_train['positions'].notnull()]

In [ ]:
# просто токенизируем предложения и приведем к нижнему регистру

In [ ]:
def tokenization(text):
    tokens = [word.lower() for word in word_tokenize(text) if word not in stops and word not in punct]
    res = []
    for token in tokens:
        if token:
            res.extend(token.split('-'))
        
    return ' '.join(res)

In [ ]:
df_train['tokenized'] = df_train['context'].apply(tokenization)

In [ ]:
def parse_positions(x):
    target_words = []
    pos = [i.split('-') for i in x.positions.split(',')]
    
    for position in pos:
        target_words.append(x.context[int(position[0]):int(position[1])-1])
    
    return target_words[0]

In [ ]:
df_train['target2'] = df_train.progress_apply(parse_positions, axis=1)

In [84]:
targets= df_train['target2'].unique().tolist()

In [118]:
df_train

,context_id,word,gold_sense_id,predict_sense_id,positions,context,context_tagged,target,context_cleaned,tokenized,target2
0,1,дар,1,3,18-22,Отвергнуть щедрый дар,отвергать_VERB щедрый_ADJ дар_NOUN,дар_NOUN,отвергать_VERB щедрый_ADJ дар_NOUN,отвергнуть щедрый дар,дар
1,2,дар,1,0,21-28,покупать преданность дарами и наградами,покупать_VERB преданность_NOUN дар_NOUN и_SCON...,дар_NOUN,покупать_VERB преданность_NOUN дар_NOUN наград...,покупать преданность дарами наградами,дарами
2,3,дар,1,0,19-23,Вот яд – последний дар моей Изоры,вот_PART яд_NOUN последний_ADJ дар_NOUN мой_DE...,дар_NOUN,яд_NOUN последний_ADJ дар_NOUN изор_NOUN,вот яд последний дар моей изоры,дар
3,4,дар,1,0,81-87,Основная функция корильных песен – повеселить ...,основной_ADJ функция_NOUN корильный_ADJ песня_...,дар_NOUN,основной_ADJ функция_NOUN корильный_ADJ песня_...,основная функция корильных песен повеселить уч...,дарам
4,5,дар,1,0,151-157,Но недели две спустя (Алевтина его когда-то об...,но_SCONJ неделя_NOUN два_NUM спустя_ADP алевти...,дар_NOUN,неделя_NOUN спустя_ADP алевтина_NOUN когда-то_...,но недели две спустя алевтина когда то просила...,даров
...,...,...,...,...,...,...,...,...,...,...,...
2068,2069,зонт,1,2,85-91,"Такая погода легко переживается весной, а вот ...",такой_DET погода_NOUN легко_ADV переживаться_V...,зонт_NOUN,погода_NOUN легко_ADV переживаться_VERB весна_...,такая погода легко переживается весной осенью ...,зонта
2069,2070,зонт,2,0,8-13,Пляжный зонт,пляжный_ADJ зонт_NOUN,зонт_NOUN,пляжный_ADJ зонт_NOUN,пляжный зонт,зонт
2070,2071,зонт,2,3,18-25,сидеть в кафе под зонтом,сидеть_VERB в_ADP кафе_NOUN под_ADP зонт_NOUN,зонт_NOUN,сидеть_VERB кафе_NOUN зонт_NOUN,сидеть кафе зонтом,зонтом
2071,2072,зонт,2,1,21-29,"Cтолики под широкими зонтами, несколько привин...",столик_NOUN под_ADP широкий_ADJ зонт_NOUN неск...,зонт_NOUN,столик_NOUN широкий_ADJ зонт_NOUN несколько_NU...,cтолики широкими зонтами несколько привинченны...,зонтами


In [85]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased', never_split=targets)
model = BertModel.from_pretrained('bert-base-multilingual-uncased', output_hidden_states=True)

In [86]:
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(105879, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
         

In [123]:
def get_vector(x):

    
    tagret_id = tokenizer.encode(x.target2, add_special_tokens=False)[0]
    
    tokens = tokenizer.encode(x.tokenized, add_special_tokens=False)
    
    if tagret_id in tokens:
        
        target_position = tokens.index(tagret_id)
        data.append((tokens, x.target2, target_position))
        
        
    else:
        raise Exception('Пустой массив!')

In [141]:
def mean_4(matrix, hdim=768, nlayers=13):

    split = np.split(matrix[:, hdim:], nlayers - 1, axis=1)

    return np.mean(np.array(split[-4:]), axis=0)


In [128]:
params = {
        'damping': 0.7,
        'max_iter': 150,
        'convergence_iter': 150,
        'preference': None,
        'affinity': 'euclidean'
    }


In [142]:
predicted = []

spectral = False


for word in tqdm(df_train['word'].unique().tolist()):
    
    n = df_train[df_train['word']==word]
    

    data=[]
    
    n.apply(get_vector, axis=1) # заполняет data
    
    
    vectors=[]

    for _tuple in data:
    

        input_ids = _tuple[0]
        
        lemmas, pos = _tuple[1], _tuple[2]
        
        outputs = model(torch.tensor(input_ids).unsqueeze(0))
        
        hidden_states = [l.detach().cpu().clone().numpy() for l in outputs[2]]

        layers= [layer[:,pos,:] for layer in hidden_states]
        
        
        vector = np.concatenate(layers, axis=1)

        vectors.append(mean_4(vector))
        
    
    
    matrix=np.vstack(vectors)
        
    
    clustering = AffinityPropagation(**params).fit(matrix)
        
    
    if spectral:
        
        nclusters = len(clustering.cluster_centers_indices_)
        
        if nclusters < 1:
            nclusters = 1
        elif nclusters == len(n):
            nclusters = 4
            
        clustering = SpectralClustering(n_clusters=4, n_init=20,
                                        assign_labels='discretize', n_jobs=2).fit(matrix)     
    
    
    cur_pred = clustering.labels_.tolist()
    
    predicted += cur_pred
    

In [144]:
df_train['predict_sense_id'] = predicted

/Users/t.zhordaniya/venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [145]:
df_train.to_csv('bert.csv', sep='\t')

# ?????

In [146]:
!python3 evaluate.py bert.csv

word	ari	count
дар	0.032451	36
двигатель	0.284169	14
двойник	0.002397	25
дворец	0.065708	13
девятка	0.022225	47
дедушка	-0.117647	9
дежурная	-0.103152	12
дежурный	0.075099	13
декабрист	0.154977	11
декрет	0.216524	12
дело	0.077767	129
демобилизация	0.006791	14
демократ	-0.036585	18
демонстрация	0.046488	37
дерево	0.094118	21
держава	-0.041940	15
дерзость	0.040280	37
десятка	0.020149	36
десяток	0.057729	20
деятель	-0.045455	14
диалог	-0.066318	14
диаметр	-0.108209	18
диплом	-0.023192	25
директор	-0.054197	11
диск	0.040095	62
дичь	-0.092369	18
длина	-0.040971	21
доброволец	0.145733	12
добыча	0.072234	35
доказательство	-0.030292	24
доктор	0.164433	17
долгота	-0.081136	13
доля	0.132302	45
дом	0.140916	36
дорога	0.070022	47
достижение	-0.004785	22
древесина	-0.074727	16
дупло	0.036344	15
дура	0.015843	12
дух	-0.031653	75
дым	0.001312	28
дымка	0.045850	18
дыхание	0.074559	55
дьявол	-0.005613	22
евро	0.000000	8
езда	0.083013	14
жаворонок	0.054054	10
жало	0.115756	11
жертва	-0.035978	37
жестоко